In [1]:
from datetime import datetime
import time
import os
from tqdm import tqdm
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import pyLDAvis.gensim_models
import jieba.posseg as jp,jieba
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib

F:\ML_ENVS\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# all_similarity = pd.read_csv('./all_similarity.txt')
# all_similarity.info()
#

In [3]:
# high_index = list(all_similarity[all_similarity['value']>=0.7].index)
#

In [4]:
# len(high_index)

In [2]:
all_sse = pd.read_csv('./all_sse.txt')
all_sse.columns=['value']
all_sse.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21622 entries, 0 to 21621
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   16294 non-null  float64
dtypes: float64(1)
memory usage: 169.0 KB


In [3]:
# all_similarity['value'].dropna()
all_sse['value'].dropna()

0        3.897729
1        3.897729
2        3.897729
3        3.897729
4        3.897729
           ...   
21617    0.000000
21618    3.293543
21619    0.000000
21620    4.241034
21621    4.327908
Name: value, Length: 16294, dtype: float64

In [ ]:


#%

In [4]:
np.mean(all_sse['value'].dropna())

3.155636859221245

In [5]:
all_sse = all_sse['value'].dropna().to_list()

In [6]:
len(all_sse)

16294

In [7]:
pd.Series(all_sse).describe()


count    16294.000000
mean         3.155637
std          1.086721
min          0.000000
25%          2.425498
50%          3.408631
75%          4.039251
max          5.625355
dtype: float64

In [8]:
min_sse = min(all_sse)
max_sse = max(all_sse)
print(min_sse, max_sse)
def min_max(x):
    return (x-min_sse)/(max_sse-min_sse)

0.0 5.625355441164328


In [9]:
all_sse[:10]

[3.897728832547578,
 3.897728832547578,
 3.897728832547578,
 3.897728832547578,
 3.897728832547578,
 3.897728832547578,
 2.912771183353776,
 2.912771183353776,
 3.4258417135141195,
 4.479643914278939]

In [11]:
all_sse = [min_max(i) for i in all_sse]
all_sse[:10]

[0.12317191230726635,
 0.12317191230726635,
 0.12317191230726635,
 0.12317191230726635,
 0.12317191230726635,
 0.12317191230726635,
 0.09204632035232901,
 0.09204632035232901,
 0.10825983367337944,
 0.14156097847799926]

In [12]:
pd.Series(all_sse).describe()

count    16294.000000
mean         0.099721
std          0.034341
min          0.000000
25%          0.076648
50%          0.107716
75%          0.127644
max          0.177767
dtype: float64

In [13]:
# count = [0]*5

# for idx, i in enumerate(all_similarity):
#     if 0<=i<0.2:
#         count[0] += 1
#     elif 0.2<=i<0.4:
#         count[1] += 1
#     elif 0.4<=i<0.6:
#         count[2] += 1
#     elif 0.6<=i<0.8:
#         count[3] += 1
#     elif 0.8<=i:
#         count[4] += 1
#
#
# count,sum(count)



# count = [0]*6
# for idx, i in enumerate(all_sse):
#     if 0<=i<1:
#         count[0] += 1
#     elif 1<=i<2:
#         count[1] += 1
#     elif 2<=i<3:
#         count[2] += 1
#     elif 3<=i<4:
#         count[3] += 1
#     elif 4<=i<5:
#         count[4] += 1
#     elif 5<=i<6:
#         count[5] += 1



# count = [0]*3
# for idx, i in enumerate(all_sse):
#     if 0<=i<2:
#         count[0] += 1
#     elif 2<=i<4:
#         count[1] += 1
#     elif 4<=i<6:
#         count[2] += 1


count = [0]*5
for idx, i in enumerate(all_sse):
    if 0<=i<0.2:
        count[0] += 1
    elif 0.2<=i<0.4:
        count[1] += 1
    elif 0.4<=i<0.6:
        count[2] += 1
    elif 0.6<=i<0.8:
        count[3] += 1
    elif 0.8<=i:
        count[4] += 1

print(count,sum(count))
print([(i/sum(count))*100 for i in count])

[16294, 0, 0, 0, 0] 16294
[100.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
with open('./commit_name.txt','r', encoding='utf-8') as f:
    lines = f.readlines()
commit_name = [i.rstrip('\n') for i in lines]

In [15]:
commit_name.index('D-CCFinder-NewData-javamelody-10-14717.java.txt')

10286

In [29]:
commit_name.index('D-CCFinder-NewData-algs4-4-5288.java.txt')

58

In [30]:
len(all_sse), all_sse.iloc[58]

(21622,
 value    0.696173
 Name: 58, dtype: float64)

In [21]:
def min_max(x):
    return (x-min_sse)/(max_sse-min_sse)
all_sse = pd.read_csv('./all_sse.txt')
all_sse.columns=['value']
all_sse.info()
all_sse['value'] = all_sse['value'].apply(lambda x: min_max(x))
high_index = list(all_sse[(all_sse['value']>=0.6) & (all_sse['value']<0.8)].index)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21622 entries, 0 to 21621
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   16294 non-null  float64
dtypes: float64(1)
memory usage: 169.0 KB


In [22]:
len(high_index)

7950

In [23]:
all_sse.describe()

,value
count,16294.000000
mean,0.560967
std,0.193183
min,0.000000
25%,0.431172
50%,0.605940
75%,0.718044
max,1.000000


In [ ]:
import shutil
src_path = 'D:/CCFinder/Commit2So(A commit will correspond to multiple sos)/'
trg_path = './Multiplexed post groups with sse greater than or equal to 0.6 and less than 0.8/'

for i in tqdm(high_index):
    file = commit_name[i]
    shutil.copyfile(src_path+file, trg_path+file)

In [2]:

files = os.listdir(trg_path)
for file in tqdm(files):
    with open(trg_path+file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = set(lines)
    with open(trg_path+file, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(line)


NameError: name 'os' is not defined

In [26]:
for file in tqdm(files):
    with open(trg_path+file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = set(lines)
        if len(lines)>=2 and len(lines)<3:
            print(file,len(lines))

100%|██████████| 8440/8440 [00:00<00:00, 9383.14it/s] 


D-CCFinder-NewData-Android-skin-support-0-1826.java.txt 2
D-CCFinder-NewData-Android-skin-support-0-1844.java.txt 2
D-CCFinder-NewData-Android-skin-support-0-1940.java.txt 2
D-CCFinder-NewData-Android-skin-support-0-1958.java.txt 2
D-CCFinder-NewData-Android-skin-support-0-1988.java.txt 2
D-CCFinder-NewData-androidannotations-2-84362.java.txt 2
D-CCFinder-NewData-androidannotations-2-85118.java.txt 2
D-CCFinder-NewData-angel-1-34157.java.txt 2
D-CCFinder-NewData-angel-1-39602.java.txt 2
D-CCFinder-NewData-angel-1-44624.java.txt 2
D-CCFinder-NewData-antlr4-2-12551.java.txt 2
D-CCFinder-NewData-cordova-android-4-16262.java.txt 2
D-CCFinder-NewData-cordova-android-5-20015.java.txt 2
D-CCFinder-NewData-databus-0-27614.java.txt 2
D-CCFinder-NewData-go-lang-idea-plugin-3-45974.java.txt 2
D-CCFinder-NewData-incubator-heron-0-30230.java.txt 2
D-CCFinder-NewData-jstorm-1-9728.java.txt 2
D-CCFinder-NewData-OpenRefine-7-49340.java.txt 2
D-CCFinder-NewData-picasso-4-5453.java.txt 2
D-CCFinder-NewD